In [1]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

def decimal_to_1hot(decimal_tuple, num_classes):
    temp_hot = list(map(lambda x: 10 ** x, decimal_tuple))
    temp_hot = list(str(item).zfill(num_classes + 1)[:-1][::-1] for item in temp_hot)
    temp_hot = list(list(item) for item in temp_hot)
    return list(list(map(int, temp_hot[i][:])) for i in range(len(temp_hot[:])))

device = torch.device('cuda:0')

In [16]:
data = pd.read_csv('wine.data', header=0)
#data = data.sample(frac = 1)
train_size = int(0.8 * len(data))
y_data = data['class'].values - 1
#y_data = np.split(data['class'].values, len(data['class'].values))
#y_data = tuple(map(int, y_data))
#y_data = decimal_to_1hot(y_data, 3)
#y_data = list(map(int, decimal_to_1hot(y_data, 3)))
#x_data = data.iloc[:,1:]
x_data = data[data.columns.difference(['class'])].values
#test_size = len(data) - train_size
#x_train, x_test = torch.utils.data.random_split(x_data, [train_size, test_size])
#y_train, y_test = torch.utils.data.random_split(y_data, [train_size, test_size])
x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.3,
    shuffle=True)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test).to(device)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test).to(device)

In [20]:
class WineNet(torch.nn.ModuleList):
    def __init__(self, n_hid_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(13, n_hid_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hid_neurons, n_hid_neurons)
        self.act2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hid_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
winenet = WineNet(13).to(device)

In [21]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(winenet.parameters(), lr=3.0e-3)

In [24]:
batch_size = 50
epochs = range(5000)

for epoch in epochs:
    order = np.random.permutation(len(x_train))
    for start_index in range(0, len(x_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index + batch_size]
        x_batch = x_train[batch_indexes].to(device)
        y_batch = y_train[batch_indexes].to(device)
        
        preds = winenet.forward(x_batch)
        loss_val = loss_func(preds, y_batch)
        loss_val.backward()
        
        optimizer.step()
    if epoch % 100 == 0:
        test_preds = winenet.forward(x_test)
        test_preds = test_preds.argmax(dim=1)
        #print(test_preds)
        #print(y_test)
        print((test_preds == y_test).float().mean())

tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0.3889, device='cuda:0')
tensor(0

In [108]:

def decimal_to_1hot(decimal_tuple):
    temp_hot = list(map(lambda x: 10 ** x, decimal_tuple))
    return [str(temp_hot[item]).zfill(len(a)+1)[:-1][::-1] for item in temp_hot]
a = (1, 2, 3)
decimal_to_1hot(a)

IndexError: list index out of range

In [68]:
a = ['1000', '0100', '0001']
c = list(a)
b = list(list(item) for item in a)
for i in range(len(b[:])):
    for j in range(len(b[:][:])):
        print(b[i][j][:])
        b[i][j][:] = int(b[i][j][:])

#c = int(b[i][j][:] for i,j in zip(range(len(b[:])),range(len(b[0][:]))))

print(b)

1


TypeError: 'str' object does not support item assignment

In [59]:
zip(range(2),range(4))

In [74]:
print(torch.randn(3, 5, requires_grad=True))

tensor([[-0.3143,  0.9649, -1.4087, -0.8164,  0.6985],
        [ 1.3983, -0.9498, -0.6232,  0.3650,  0.3820],
        [ 0.9345, -0.1012, -0.7430, -0.9989, -0.0268]], requires_grad=True)


In [75]:
print(torch.randn(3, 5))

tensor([[ 0.0926, -0.5898,  0.5126,  0.8062, -1.7700],
        [ 0.0914, -0.0754,  0.8393,  0.2479,  0.2067],
        [ 0.9928, -0.8986, -1.2028,  3.1501, -0.2165]])


In [110]:
import sklearn